## Advertising search on Avito website

В этом задании вам нужно будет написать функцию, которая парсит [Авито](https://www.avito.ru) (будем рассматривать только Москву). Эта функция принимает два параметра:
* search - то, что мы ищем на avito
* page - номер страницы, с которой парсим информацию

```python
def parse_page(search, page):
    # тут ваш код
df = parse_page('гречка', 4)
```
Нужно скачать следующую информацию:
* название объявления
* ссылка на объявление
* цена
* метро (если есть), вам нужно аккуратно обрабатывать None или использовать конструкцию try-except
* сколько метров от метро (если есть)

Эту информацию лучше сохранить в виде pandas df.

В данном задании **не нужно** парсить все страницы (вас могут забанить). Нужно лишь сделать функцию, которая парсит одну страницу. Чтобы вас не забанили сделаейте request один раз и работайте с вашей страничкой без функции. После того как все будет готово, оберните весь ваш код в функцию.

In [120]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # регулярные выражения
import pandas as pd 
import numpy as np

In [142]:
def parsing(search, page):
    h3s = BeautifulSoup(requests.get(f"https://www.avito.ru/moskva_i_mo?q={search}").text, "html5lib").find_all("h3", limit=page)
    h3new = []
    h3new.append(h3s[page-1])
    title = re.findall('title="(.*)"', str(h3new))
    links=["https://www.avito.ru/" + l.find_all("a")[0]["href"] for l in h3new]
    txt = BeautifulSoup(requests.get(links[0]).text, "html.parser").find_all("div")
    price = re.findall('<span class="js-item-price" content="(\d*)"',str(txt))
    price = price[0]
    try:
        for i in range(len(txt)):
            subway = txt[i].find("span", {"class": "item-address-georeferences-item__content"})
            meters = txt[i].find("span", {"class": "item-address-georeferences-item__after"})
            if subway != None and meters != None:
                break
    except:
        subway = np.nan
        meters = np.nan
    if subway is not np.nan:
        subway = re.findall('>(.*)</span>', str(subway))  
        subway = subway[0]
    if meters is not np.nan:
        meters = re.findall('>(.*)</span>', str(meters))
        meters = meters[0].replace("\xa0", " ")
    
    data = {'title': title, 'url': links, 'price': price, 'subway': subway , 'meters': meters}
    df = pd.DataFrame(data = data)
    return df


In [347]:
df = parsing('жесткий диск', 2)

In [348]:
df

,title,url,price,subway,meters
0,Жесткий диск 1 TB WD1003fbyx в Москве,https://www.avito.ru//moskva/tovary_dlya_kompy...,1500,Комсомольская,100 м


In [ ]:
df = parsing('гречка', 2)

In [150]:
df

,title,url,price,subway,meters
0,Гречка оптом в Красногорске,https://www.avito.ru//moskovskaya_oblast_krasn...,49,Мякинино,1 км


In [151]:
df = parsing('кровать', 3)
df

,title,url,price,subway,meters
0,Кровать с матрасом Аскона.Новое в упаковке в М...,https://www.avito.ru//moskva/mebel_i_interer/k...,9500,Щелковская,"1,9 км"


### Если есть станция метро, то показывается ближайшая.Если станции нет, то `NaN` 